In [63]:

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
Xtrain = pd.read_csv('./X_train.csv')
res = pd.read_csv('./y_train.csv')
Xtrain["response"] = res["y"]
Xtest = pd.read_csv('./X_test.csv')
me = res["y"].mean()
sd1 = res["y"].std()
Xtest = (Xtest - Xtrain.mean())/Xtrain.std()
Xtrain = (Xtrain - Xtrain.mean())/Xtrain.std()

# Define the neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(21, 15) # input layer to first hidden layer
        self.fc1p = nn.Linear(15, 18) # input layer to first hidden layer
        self.fc2 = nn.Linear(18, 13) # first hidden layer to second hidden layer
        self.fc2p = nn.Linear(13, 21) # first hidden layer to second hidden layer
        self.fc3 = nn.Linear(21, 1) # second hidden layer to output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc1p(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2p(x))
        x = self.fc3(x)
        return x

# Initialize the model
model = Net()

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
my_list= [Xtrain["BMXHT"],
          Xtrain["BMXWT"],
          Xtrain["RIAGENDR"],
          Xtrain["DR1TSUGR"],
          Xtrain["DMDBORN4"],
          Xtrain["SMAQUEX2"],
          Xtrain["BPACSZ"],
          Xtrain["DMDHRGND"],
          Xtrain["DR1TTFAT"],
          Xtrain["BMXARML"],
          Xtrain["DMDHHSIZ"],
          Xtrain["BMXWAIST"],
          Xtrain["BMXSAD1"],
          Xtrain["LBDTCSI"],
          Xtrain["DR1TALCO"],
          Xtrain["BPXPLS"],
          Xtrain["BMXBMI"],
          Xtrain["BMXLEG"],
          Xtrain["BPXPTY"],
          Xtrain["WTMEC2YR"],
          Xtrain["DR1BWATZ"]]
# Train the model
n_epoch = 10000
for epoch in range(n_epoch):
    # Forward pass
    inputs = torch.Tensor(np.array(my_list)).T # Transpose the input tensor
    labels = torch.Tensor(Xtrain["response"].values).view(-1, 1)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    # Backward and optimize
    loss.backward()
    optimizer.step()
    
    # Print progress
    if epoch % 1000 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, n_epoch, loss.item()))

# Save the model
torch.save(model.state_dict(), 'my_model.pt')

Epoch [1/10000], Loss: 1.0183
Epoch [1001/10000], Loss: 0.5452
Epoch [2001/10000], Loss: 0.4439
Epoch [3001/10000], Loss: 0.3933


In [62]:
# read in the CSV file


# convert the test data to a PyTorch tensor
my_list2= [Xtest["BMXHT"],
          Xtest["BMXWT"],
          Xtest["RIAGENDR"],
          Xtest["DR1TSUGR"],
          Xtest["DMDBORN4"],
          Xtest["SMAQUEX2"],
          Xtest["BPACSZ"],
          Xtest["DMDHRGND"],
          Xtest["DR1TTFAT"],
          Xtest["BMXARML"],
          Xtest["DMDHHSIZ"],
          Xtest["BMXWAIST"],
          Xtest["BMXSAD1"],
          Xtest["LBDTCSI"],
          Xtest["DR1TALCO"],
          Xtest["BPXPLS"],
          Xtest["BMXBMI"],
          Xtest["BMXLEG"],
          Xtest["BPXPTY"],
          Xtest["WTMEC2YR"],
          Xtest["DR1BWATZ"]]
Xtest_tensor = torch.from_numpy(np.array(my_list2).T).float()

ysimp = pd.read_csv("./y_sample.csv")
seq = ysimp["SEQN"]
# make predictions on the test data using the trained model
predictions = model(Xtest_tensor)

# convert the predictions to a numpy array
predictions = (predictions.detach().numpy() * sd1) + me

# save the predictions to a CSV file

prediction = pd.DataFrame()
prediction['SEQN']=seq 
prediction['y']=predictions
prediction.to_csv('prediction.csv')